# Verify Credential

This notebook illustrates how a VC signed using a BIP 322 signature could be parsed and verified by a verifier.

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and [Digital Contract Design](https://contract.design/). Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

## Steps

1. Parse signature from signed VC
2. Parse/resolve bitcoin address used the sign vc
3. Remove proof from signed VC and canonicalize
4. Verify signatue on VC

In [1]:
signed_vc = {'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'https://example.edu/issuers/565049',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}},
 'proof': {'type': 'BIP322SimpleSignature2022',
  'created': '2022-05-23T21:19:10Z',
  'proofPurpose': 'assertionMethod',
  'verificationMethod': 'https://example.edu/issuers/565049#key-1',
  'bip322_sig': 'AkgwRQIhAKgZ4oVu51lwBxQ+u0yz1xHBg60ovueTTvU8Wl5RkrhoAiAfY5WS+ghkFmQZzhLcGN6wSAjlQeLbBYgPi1Ve3hvnswEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x'}}

In [2]:
proof = signed_vc["proof"]
bip322_sig = proof["bip322_sig"]

In [3]:
vc = signed_vc
del vc["proof"]

vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'https://example.edu/issuers/565049',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}}}

In [4]:
import json
from buidl.helper import str_to_bytes

In [5]:
vc_payload = json.dumps(vc)

In [6]:
str_to_bytes(vc_payload)

b'{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d\'Universit\\u00e9", "lang": "fr"}]}}}'

In [7]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


In [8]:
## TODO: this should be contained within the VC or retrievable through did resolution.
## Currently just copied from issue_credential
address = 'bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck'

In [9]:
from buidl.script import address_to_script_pubkey
address_to_script_pubkey(address)

OP_0 15142887254a220330b3d12e644bc3c4cee83020 

In [10]:
from src.message import verify_message

In [11]:
verify_message(address, bip322_sig, vc_payload)

mismatch between length and consumed bytes 88 vs 31


True